# Pedri Detection with YOLOv8

This notebook detects FC Barcelona's Pedri in match footage using YOLOv8, an upgrade from YOLOv5 for better accuracy and speed. It includes data augmentation, model training, and video detection in one educational workflow.

## Overview
- **Purpose**: Detect and track Pedri in a Barcelona game video.
- **Tools**: YOLOv8 (Ultralytics), Albumentations, OpenCV.
- **Status**: Functional, with room for refinement.

## Requirements
- GPU-enabled environment (e.g., Kaggle T4).
- Adjust paths for non-Kaggle use.

## 1. Data Augmentation

Augments the training dataset.

In [1]:
import albumentations as A
import cv2
import os

# Define transformation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Blur(blur_limit=3, p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Input directories (read-only)
input_img_dir = "/kaggle/input/pedri-dataset/images/train"
input_label_dir = "/kaggle/input/pedri-dataset/labels/train"

# Output directories (writable)
output_img_dir = "/kaggle/working/pedri-dataset/images/train"
output_label_dir = "/kaggle/working/pedri-dataset/labels/train"

# Create output directories
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Process each image
for img_file in os.listdir(input_img_dir):
    if not img_file.endswith('.jpg'):
        continue
    
    img_path = os.path.join(input_img_dir, img_file)
    label_path = os.path.join(input_label_dir, img_file.replace('.jpg', '.txt'))
    
    image = cv2.imread(img_path)
    if image is None:
        print(f"Failed to load image: {img_path}")
        continue
    
    bboxes = []
    classes = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                label, x, y, w, h = map(float, line.split())
                bboxes.append([x, y, w, h])
                classes.append(int(label))
    else:
        print(f"Label file not found: {label_path}")
        continue

    augmented = transform(image=image, bboxes=bboxes, class_labels=classes)
    
    aug_img_path = os.path.join(output_img_dir, f"aug_{img_file}")
    aug_label_path = os.path.join(output_label_dir, f"aug_{img_file.replace('.jpg', '.txt')}")
    
    cv2.imwrite(aug_img_path, augmented['image'])
    with open(aug_label_path, "w") as f:
        for cls, bbox in zip(augmented['class_labels'], augmented['bboxes']):
            f.write(f"{cls} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")
    
    print(f"Augmented and saved: {aug_img_path}")

## 2. Setup YOLOv8

Installs YOLOv8 via Ultralytics package.

In [2]:
!pip install ultralytics
from ultralytics import YOLO

## 3. Define Training Parameters

Sets variables for training.

In [3]:
PROJECT_NAME = "pedri_detection"
BASE_MODEL = "yolov8l.pt"  # Upgraded to YOLOv8 large model
TRAIN_BATCH = 64
TRAIN_EPOCHS = 100
VAL_BATCH = 64

## 4. Create Data Configuration

Generates `data.yaml`.

In [4]:
import yaml

train_path = "/kaggle/input/pedri-dataset/images/train"
val_path = "/kaggle/input/pedri-dataset/images/val"
test_path = "/kaggle/input/pedri-dataset/images/test"

with open("data.yaml", "w") as file:
    yaml.dump({
        "train": train_path,
        "val": val_path,
        "test": test_path,
        "nc": 2,
        "names": ["Pedri", "Others"]
    }, stream=file, default_flow_style=None)

## 5. Train the Model

Trains YOLOv8 with frozen layers.

In [5]:
model = YOLO(BASE_MODEL)
model.train(data="data.yaml", epochs=TRAIN_EPOCHS, batch=TRAIN_BATCH, imgsz=640, project=PROJECT_NAME, name="feature_extraction", freeze=12)

## 6. Detect Pedri in Video

Runs detection on `barca-video.mp4`.

In [6]:
model = YOLO("/kaggle/working/pedri_detection/feature_extraction/weights/best.pt")
results = model.predict(source="/kaggle/input/pedri-dataset/barca-video.mp4", imgsz=640, conf=0.25, save=True, save_txt=True, save_conf=True, line_thickness=3, project="detect-runs", name="pedri-detect")

## Notes
- **Paths**: Kaggle-specific. For local use, update to your directories.
- **Output**: Video saved to `detect-runs/pedri-detect/barca-video.mp4`.
- **Improvements**: Consider YOLOv9/YOLOv10 for further gains (see README).